In [1]:
from datasets import load_dataset

# Load private dataset
dataset = load_dataset("mikeriess/LM_DA_test1")

README.md:   0%|          | 0.00/499 [00:00<?, ?B/s]

c:\Users\miker\anaconda3\envs\LMLLM\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\miker\.cache\huggingface\hub\datasets--mikeriess--LM_DA_test1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/90.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/378 [00:00<?, ? examples/s]

In [2]:
# Convert to pandas and then to dictionary format
df = dataset['train'].to_pandas()
df

,question_id,model_generator,conversation,language,turn
0,90dd550e82a043089afa82f0b9c6b401,meta-llama/llama-3.3-70b-instruct,[{'content': 'Er der en måde at undgå forsinke...,Danish,1
1,90dd550e82a043089afa82f0b9c6b401,meta-llama/llama-3.3-70b-instruct,"[{'content': 'Det lyder som en god idé, men hv...",Danish,2
2,90dd550e82a043089afa82f0b9c6b401,meta-llama/llama-3.3-70b-instruct,"[{'content': 'Det lyder interessant, men hvord...",Danish,3
3,90dd550e82a043089afa82f0b9c6b401,meta-llama/llama-3.3-70b-instruct,"[{'content': 'Okay, jeg forstår. Så det er en ...",Danish,4
4,5314e30667a24f3abd327c6cc3efa646,meta-llama/llama-3.3-70b-instruct,[{'content': 'Hvordan kan jeg overføre min vin...,Danish,1
...,...,...,...,...,...
373,0d64d5a2ecba45c8a5f3656f2993067c,meta-llama/llama-3.3-70b-instruct,[{'content': 'Det er en god idé! Jeg skal nok ...,Danish,4
374,13a1417f6e9146ae8704baf007e5fc88,meta-llama/llama-3.3-70b-instruct,"[{'content': 'Har du nogensinde oplevet, at di...",Danish,1
375,13a1417f6e9146ae8704baf007e5fc88,meta-llama/llama-3.3-70b-instruct,[{'content': 'Det giver mening. Jeg har selv o...,Danish,2
376,13a1417f6e9146ae8704baf007e5fc88,meta-llama/llama-3.3-70b-instruct,"[{'content': 'Okay, det lyder som en god plan....",Danish,3


In [6]:
df.loc[df.question_id == "90dd550e82a043089afa82f0b9c6b401"]['conversation']

0    [{'content': 'Er der en måde at undgå forsinke...
1    [{'content': 'Det lyder som en god idé, men hv...
2    [{'content': 'Det lyder interessant, men hvord...
3    [{'content': 'Okay, jeg forstår. Så det er en ...
Name: conversation, dtype: object

In [12]:
# Convert dataset to pandas
df = dataset['train'].to_pandas()

# Group by question_id and concatenate conversations
chatml_data = {}
for question_id in df['question_id'].unique():
    # Get all rows for this question_id
    question_rows = df[df['question_id'] == question_id]
    
    # Sort by order if needed
    question_rows = question_rows.sort_values('turn')
    
    # Create messages list
    messages = []
    for _, row in question_rows.iterrows():
        # Assuming each conversation entry is a list of dicts with 'content' key
        for message in row['conversation']:
            messages.append({
                "role": "user" if len(messages) % 2 == 0 else "assistant",
                "content": message['content']
            })
    
    # Store in chatml format
    chatml_data[question_id] = {
        "messages": messages,
        "language": question_rows.iloc[0]['language']  # Take language from first row
    }

chatml_data

{'90dd550e82a043089afa82f0b9c6b401': {'messages': [{'role': 'user',
    'content': 'Er der en måde at undgå forsinkelse på nye indspilninger, når man har mange plug-ins kørende på andre spor i et projekt?'},
   {'role': 'assistant',
    'content': 'Ja, det er et godt spørgsmål! En måde at løse det på er at bounce alle spor med plug-ins til nye spor, så du kan indspille nye tracks uden forsinkelse. Eller også kan du oprette en ny session til kun vokalindspilning og importere de bounced spor derind.'},
   {'role': 'user',
    'content': 'Det lyder som en god idé, men hvad med hvis jeg har mange spor med plug-ins, og jeg ikke ønsker at bounce dem alle? Er der en anden måde at undgå forsinkelsen?'},
   {'role': 'assistant',
    'content': 'Du kunne også overveje at bruge en funktion som "Remove Plugins" i Audio Configuration, når du skal indspille nye tracks. Det vil fjerne alle plug-ins fra projektet, så du kan indspille uden forsinkelse, og derefter kan du importere de nye tracks tilbage

In [15]:

print("Sample conversation:")
sample_id = list(chatml_data.keys())[1]
print(f"\nQuestion ID: {sample_id}")
print(f"Language: {chatml_data[sample_id]['language']}")
print("\nMessages:")
for msg in chatml_data[sample_id]['messages']:
    #print(f"{msg['role']}: {msg['content'][:100]}...")
    print(f"{msg['role']}: {msg['content']}"+ "\n")

Sample conversation:

Question ID: 5314e30667a24f3abd327c6cc3efa646
Language: Danish

Messages:
user: Hvordan kan jeg overføre min vinyl-samling til min computer, så jeg kan lytte til dem digitalt?

assistant: Det er faktisk ret let! Du skal bruge en phono-til-USB-forstærker eller et lydkort med en phono-indgang. Dette vil tillade dig at tilslutte din pladespiller direkte til computeren og indspille lyden.

user: Jeg har prøvet at bruge et almindeligt kabel fra min pladespiller til computeren, men der kom meget støj med. Er det måske mit kabel, der er problemet?

assistant: Det kan være muligt, men det er mere sandsynligt, at problemet ligger i, at du mangler en RIAA-forstærker. Disse forstærkere er nødvendige for at korrekt indspille lyd fra en pladespiller, da de kan kompensere for den specifikke frekvenskurve, som vinyl-plader bruger.

user: Hvad er en RIAA-forstærker, og hvordan kan jeg få fat i en?

assistant: En RIAA-forstærker er en lille enhed, der er designet til at forstærke 